## Task 1: MLP


In [11]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


In [19]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mlp = MLP().to(device)


In [20]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0


print('Finished Training MLP')


[1,  2000] loss: 1.935
[1,  4000] loss: 1.717
[1,  6000] loss: 1.663
[1,  8000] loss: 1.617
[1, 10000] loss: 1.588
[1, 12000] loss: 1.557
[2,  2000] loss: 1.477
[2,  4000] loss: 1.492
[2,  6000] loss: 1.461
[2,  8000] loss: 1.463
[2, 10000] loss: 1.465
[2, 12000] loss: 1.446
[3,  2000] loss: 1.361
[3,  4000] loss: 1.378
[3,  6000] loss: 1.347
[3,  8000] loss: 1.362
[3, 10000] loss: 1.381
[3, 12000] loss: 1.389
[4,  2000] loss: 1.287
[4,  4000] loss: 1.297
[4,  6000] loss: 1.310
[4,  8000] loss: 1.301
[4, 10000] loss: 1.306
[4, 12000] loss: 1.307
[5,  2000] loss: 1.226
[5,  4000] loss: 1.257
[5,  6000] loss: 1.247
[5,  8000] loss: 1.244
[5, 10000] loss: 1.261
[5, 12000] loss: 1.269
Finished Training MLP


In [22]:
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = mlp(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
print(f'Average test loss: {test_loss / len(testloader)}')


Accuracy of the network on the 10000 test images: 50 %
Average test loss: 1.415122298401594


The MLP model achieves an accuracy of 50% on the CIFAR-10 test set with average test loss of 1.415, it means that it correctly classified half of the images in the test set.

An accuracy of 49% is relatively low for the CIFAR-10 dataset. The CIFAR-10 dataset is challenging due to its small image size (32x32 pixels) and diverse range of object categories. Achieving higher accuracies usually requires more sophisticated models or techniques.

## Task 2: CNN

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Using device: cuda:0


100%|██████████| 170498071/170498071 [00:01<00:00, 87606887.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

cnn = SimpleCNN().to(device)


In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)


In [4]:
for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 2.204
[1,  4000] loss: 1.883
[1,  6000] loss: 1.686
[1,  8000] loss: 1.595
[1, 10000] loss: 1.528
[1, 12000] loss: 1.467
[2,  2000] loss: 1.377
[2,  4000] loss: 1.351
[2,  6000] loss: 1.327
[2,  8000] loss: 1.306
[2, 10000] loss: 1.274
[2, 12000] loss: 1.246
[3,  2000] loss: 1.169
[3,  4000] loss: 1.176
[3,  6000] loss: 1.175
[3,  8000] loss: 1.166
[3, 10000] loss: 1.156
[3, 12000] loss: 1.169
[4,  2000] loss: 1.054
[4,  4000] loss: 1.080
[4,  6000] loss: 1.071
[4,  8000] loss: 1.064
[4, 10000] loss: 1.050
[4, 12000] loss: 1.076
[5,  2000] loss: 0.994
[5,  4000] loss: 1.012
[5,  6000] loss: 0.997
[5,  8000] loss: 0.978
[5, 10000] loss: 1.011
[5, 12000] loss: 1.000
Finished Training


In [5]:
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
print(f'Average test loss: {test_loss / len(testloader)}')


Accuracy of the network on the 10000 test images: 64 %
Average test loss: 1.03826994513534


An accuracy of 64% indicates that the CNN model performed better than the MLP model on the CIFAR-10 dataset. CNNs are well-suited for image classification tasks due to their ability to capture spatial hierarchies of features, which might have contributed to the improved performance.

Even though the CNN model achieved 64% accuracy with average loss of 1.038, there may still be room for improvement.

## Task 3: VGG

In [23]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Using device: cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [24]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False

num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1]
features.extend([nn.Linear(num_features, len(classes))])
vgg16.classifier = nn.Sequential(*features)

vgg16 = vgg16.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)


In [9]:
for epoch in range(5):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


Finished Training


In [10]:
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')
print(f'Average test loss: {test_loss / len(testloader)}')


Accuracy of the network on the 10000 test images: 83.75 %
Average test loss: 0.47695332286274356


The pretrained VGG16 model achieves an accuracy of 83.75% and average test loss of 0.476 on the CIFAR-10 test set, it demonstrates strong performance and significant improvement over both the MLP and CNN models.

This suggests that leveraging transfer learning from a pre-trained model like VGG16, which was originally trained on a large dataset like ImageNet, can lead to superior performance compared to training models from scratch.

Transfer learning allows the model to leverage the knowledge learned from a large dataset (ImageNet) to extract meaningful features relevant to the CIFAR-10 classification task. The pretrained VGG16 model has learned to recognize a wide range of low-level and high-level features, which benefits the classification of CIFAR-10 images.

The high accuracy achieved by the pretrained VGG16 model suggests strong generalization ability.